In [1]:
import requests
from bs4 import BeautifulSoup
import reverse_geocode
import folium
from geopy.geocoders import Nominatim
import numpy as np
import matplotlib as mpl
import pandas as pd
from sklearn import datasets
import geocoder

In [2]:
Raleigh_Neighborhoods=[]
response_obj=requests.get('https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Raleigh,_North_Carolina').text
soup=BeautifulSoup(response_obj,'lxml')
mydivs = soup.findAll("div", {"class": "mw-category-group"})
for div in mydivs:
    for l in div.findAll('a'):
        for title in l:
            Raleigh_Neighborhoods.append(title)
Raleigh_Neighborhoods.remove('Raleigh, North Carolina neighborhoods')
Raleigh_Neighborhoods=[s.replace('Historic District','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.replace('(Raleigh, North Carolina)','') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s.strip(' ') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods=[s+(',Raleigh, North Carolina') for s in Raleigh_Neighborhoods]
Raleigh_Neighborhoods

['Battery Heights,Raleigh, North Carolina',
 'Bloomsbury,Raleigh, North Carolina',
 'Boylan Heights,Raleigh, North Carolina',
 'Cameron Park,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Cameron Village,Raleigh, North Carolina',
 'Capitol Area,Raleigh, North Carolina',
 'Capitol Heights,Raleigh, North Carolina',
 'Centennial Campus of North Carolina State University,Raleigh, North Carolina',
 'Depot,Raleigh, North Carolina',
 'East Raleigh–South Park,Raleigh, North Carolina',
 'Fayetteville Street (Raleigh),Raleigh, North Carolina',
 'Fayetteville Street,Raleigh, North Carolina',
 'Five Points Historic Neighborhoods,Raleigh, North Carolina',
 'Glenwood Avenue,Raleigh, North Carolina',
 'Glenwood–Brooklyn,Raleigh, North Carolina',
 'Hayes Barton,Raleigh, North Carolina',
 'Hi-Mount,Raleigh, North Carolina',
 'Historic Oakwood,Raleigh, North Carolina',
 'Longview Gardens,Raleigh, North Carolina',
 'Madonna Acres,Raleigh, North Carolina',
 'Maiden Lane,Raleigh, N

In [3]:
df_ral=pd.DataFrame(Raleigh_Neighborhoods)
df_ral=df_ral.rename(columns={0:'Neighborhoods'})
df_ral=df_ral.drop_duplicates(keep='first')
df_ral

,Neighborhoods
0,"Battery Heights,Raleigh, North Carolina"
1,"Bloomsbury,Raleigh, North Carolina"
2,"Boylan Heights,Raleigh, North Carolina"
3,"Cameron Park,Raleigh, North Carolina"
4,"Cameron Village,Raleigh, North Carolina"
6,"Capitol Area,Raleigh, North Carolina"
7,"Capitol Heights,Raleigh, North Carolina"
8,Centennial Campus of North Carolina State Univ...
9,"Depot,Raleigh, North Carolina"
10,"East Raleigh–South Park,Raleigh, North Carolina"


In [4]:
geolocator=Nominatim(user_agent='Raleigh')
city=[]
lat=[]
long=[]
from geopy.exc import GeocoderTimedOut
try:
    for i in Raleigh_Neighborhoods:
        location = geolocator.geocode(i)
        if location is not None:
            city.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [5]:
df=pd.DataFrame({'Neighborhoods':city,'latitude':lat,'longitude':long})

In [6]:
df=df.drop_duplicates(subset='latitude',keep='first')

In [7]:
df

,Neighborhoods,latitude,longitude
0,"Battery Heights,Raleigh, North Carolina",35.777058,-78.617563
1,"Bloomsbury,Raleigh, North Carolina",35.808897,-78.648599
2,"Boylan Heights,Raleigh, North Carolina",35.774159,-78.652102
3,"Cameron Park,Raleigh, North Carolina",35.785779,-78.655470
4,"Cameron Village,Raleigh, North Carolina",35.790361,-78.660413
6,"Capitol Area,Raleigh, North Carolina",35.789884,-78.621851
8,"Depot,Raleigh, North Carolina",35.780398,-78.639099
9,"Fayetteville Street (Raleigh),Raleigh, North C...",35.768875,-78.641336
11,"Glenwood Avenue,Raleigh, North Carolina",35.886724,-78.744499
12,"Hayes Barton,Raleigh, North Carolina",35.802755,-78.646583


In [19]:
raleigh=['North Raleigh,Raleigh,nc','Southwest raleigh,Raleigh,nc','Southeast raleigh,Raleigh,nc'
        ,'east raleigh,Raleigh,nc']

In [20]:
raliegh_side=[]
lat=[]
long=[]
from geopy.exc import GeocoderTimedOut
try:
    for i in raleigh:
        location = geolocator.geocode(i)
        if location is not None:
            raliegh_side.append(i)
            lat.append(location.latitude)
            long.append(location.longitude)
except GeocoderTimedOut as e:
    print("Error: geocode failed on input %s with message %s"%(location, e.msg))

In [21]:
df_ral=pd.DataFrame({'Raleigh_Side':raliegh_side,'latitude':lat,'longitude':long})
df_ral

,Raleigh_Side,latitude,longitude
0,"North Raleigh,Raleigh,nc",35.908068,-78.600658
1,"Southwest raleigh,Raleigh,nc",35.773076,-78.702732
2,"Southeast raleigh,Raleigh,nc",35.768612,-78.596097
3,"east raleigh,Raleigh,nc",37.756782,-81.174266


In [32]:
CLIENT_ID = 'EA0WZQZP3HJVT4DTV02NLEGNIFSAHPTRXK50TAZIIYY5A5OG' # your Foursquare ID
CLIENT_SECRET = '5X1OY3KPXNEM1F5CVU2A124DQUNH3GC0TDP1MP1M0FX31D1Z' # your Foursquare Secret

# CLIENT_ID ='BCZ1DKRCRZTMRZPSY5HHEQTQJNZCITUWFO3CLRCIUQOOGSFB'
# CLIENT_SECRET ='WPWOCZ3QZVXYQCXPKBPKSKW1CQSFBLTKAWYGSS0WBXJKZZZW'
VERSION = '20190104'
LIMIT = 150
radius= 5000
latitude = 35.787743
longitude = -78.644257
search_query='Restaurants'
restID=[]
restName=[]
restCity=[]
restLat=[]
restLng=[]
restCat=[]

In [35]:
url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude,longitude,VERSION,search_query,radius,LIMIT)
results = requests.get(url1).json()
r=results['response']['groups'][0]['items']
for a in range(len(r)):
     print(r[a]['venue']['name'],r[a]['venue']['name'])

In [55]:
# for lat ,lng in zip(df_ral['latitude'],df_ral['longitude']):  
#     url1 = 'https://api.foursquare.com/v2/venues/timeseries?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng, VERSION)
#     results = requests.get(url1).json()
#     print(results)

In [58]:
restCity_Lat=[]
restCity_Lng=[]
restName_Ral=[]
restCity_Ral=[]
restLat_Ral=[]
restLng_Ral=[]
restCat_Ral=[]
for neigh,lat ,lng in zip(df_ral['Raleigh_Side'],df_ral['latitude'],df_ral['longitude']):  
    url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng,VERSION,search_query,radius,LIMIT)
    results = requests.get(url1).json()
    r=results['response']['groups'][0]['items']
    for a in range(len(r)):   
        restCity_Ral.append(neigh)
        restCity_Lat.append(lat)
        restCity_Lng.append(lng)
        restName_Ral.append(r[a]['venue']['name'])
        restCat_Ral.append(r[a]['venue']['categories'][0]['name'])
        restLat_Ral.append(r[a]['venue']['location']['lat'])
        restLng_Ral.append(r[a]['venue']['location']['lng'])

#print(neigh,r[a]['venue']['name'],r[a]['venue']['categories'][0]['name'],r[a]['venue']['location']['lat'],r[a]['venue']['location']['lng'])

In [63]:
final_Ral=pd.DataFrame(list(zip(restCity_Ral,restCity_Lat,restCity_Lng,restName_Ral,restCat_Ral,restLat_Ral,restLng_Ral)),columns=['Neighbourhood','Neigh_Lat','Neigh_Lng','Venue','Venue_Category','Latitude','Longitude'])
final_Ral

,Neighbourhood,Neigh_Lat,Neigh_Lng,Venue,Venue_Category,Latitude,Longitude
0,"North Raleigh,Raleigh,nc",35.908068,-78.600658,City Barbeque,BBQ Joint,35.901950,-78.602888
1,"North Raleigh,Raleigh,nc",35.908068,-78.600658,Bella Italia,Pizza Place,35.906059,-78.592818
2,"North Raleigh,Raleigh,nc",35.908068,-78.600658,Bad Daddy's Burger Bar,Burger Joint,35.901833,-78.603660
3,"North Raleigh,Raleigh,nc",35.908068,-78.600658,Bojangles' Famous Chicken 'n Biscuits,Restaurant,35.905882,-78.591228
4,"North Raleigh,Raleigh,nc",35.908068,-78.600658,Nashers,Sandwich Place,35.906080,-78.592399
...,...,...,...,...,...,...,...
332,"east raleigh,Raleigh,nc",37.756782,-81.174266,Pizza Hut,Pizza Place,37.790621,-81.210696
333,"east raleigh,Raleigh,nc",37.756782,-81.174266,Wendy's,Fast Food Restaurant,37.749778,-81.137964
334,"east raleigh,Raleigh,nc",37.756782,-81.174266,Arby's,Fast Food Restaurant,37.797292,-81.183825
335,"east raleigh,Raleigh,nc",37.756782,-81.174266,Tudor's Biscuit World,Comfort Food Restaurant,37.752894,-81.219827


In [54]:
final_Ral.groupby(['Category']).count()

,Neighbourhood,Name,Latitude,Longitude
Category,,,,
American Restaurant,24,24,24,24
Arepa Restaurant,1,1,1,1
Asian Restaurant,9,9,9,9
BBQ Joint,8,8,8,8
Bagel Shop,5,5,5,5
Bakery,10,10,10,10
Breakfast Spot,9,9,9,9
Burger Joint,11,11,11,11
Burrito Place,2,2,2,2


In [57]:
# for neigh,lat ,lng in zip(df['Neighborhoods'],df['latitude'],df['longitude']):  
#     url1 = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, lat,lng,VERSION,search_query,radius,LIMIT)
#     results = requests.get(url1).json()
#     r=results['response']['groups'][0]['items']
#     for a in range(len(r)):
#         restCity.append(neigh)
#         restName.append(r[a]['venue']['name'])
#         restCat.append(r[a]['venue']['categories'][0]['name'])
#         restLat.append(r[a]['venue']['location']['lat'])
#         restLng.append(r[a]['venue']['location']['lng'])        
#         #print(neigh,r[a]['venue']['name'],r[a]['venue']['categories'][0]['name'],r[a]['venue']['location']['lat'],r[a]['venue']['location']['lng'])
#         #print(r[a]['venue']['name'],r[a]['venue']['categories'][0]['name'],r[a]['venue'])
         

In [56]:
# final=pd.DataFrame(list(zip(restCity,restName,restCat,restLat,restLng)),columns=['Neighbourhood','Name','Category','Latitude','Longitude'])
# final.sort_values(by=['Name'])
# final.shape